In [1]:
from probemapper.registration import ANTSPointTransformation, ANTSTransformationStages
from probemapper.io import save_nifti
import os, subprocess
import numpy as np
import pandas as pd

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220526_P190_laqueus_DiI_L-F11/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, f"indexing")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Transform channel coordinates

In [4]:
T = ANTSPointTransformation(
    ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin/",
    num_threads=8
)
T.set_transformations([
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_0GenericAffine.mat"), inverse=True),
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_1InverseWarp.nii.gz"), inverse=False),
])
T.set_moving_points(os.path.join(rootdir, "analysis/skeleton/channel_skeleton.csv"))
T.set_outname(os.path.join(tmpdir, "channel_LUT.csv"))

In [5]:
T.run()

['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsApplyTransformsToPoints', '-d', '3', '-i', '/bucket/ReiterU/DBS/data/20220526_P190_laqueus_DiI_L-F11/analysis/skeleton/channel_skeleton.csv', '-o', '/flash/ReiterU/tmp/P190/indexing/channel_LUT.csv', '-t', '[/bucket/ReiterU/DBS/data/20220526_P190_laqueus_DiI_L-F11/analysis/ants/F2M_0GenericAffine.mat,1]', '-t', '/bucket/ReiterU/DBS/data/20220526_P190_laqueus_DiI_L-F11/analysis/ants/F2M_1InverseWarp.nii.gz']



### Query region ID

In [6]:
from probemapper import skeletonization

In [7]:
df = pd.read_csv(os.path.join(tmpdir, "channel_LUT.csv"))
coords = np.array([df["Z"], df["Y"], df["X"]])

In [8]:
regions = skeletonization.query_region_id2(coords,
                     "/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                     100, coords.shape[1]-100, 10)

In [9]:
df["Region"] = regions

In [10]:
df.insert(0, "channel", df.pop("channel"))

In [11]:
display(df)

,channel,X,Y,Z,X_org,Y_org,Z_org,Region
0,-50,1810.84,1893.350,556.801,1594.30,3144.690,764.069,3
1,-49,1812.64,1892.670,568.663,1594.31,3141.500,773.037,3
2,-48,1814.44,1891.940,580.563,1594.33,3138.320,782.006,3
3,-47,1816.24,1891.130,592.517,1594.34,3135.130,790.974,3
4,-46,1818.04,1890.250,604.528,1594.35,3131.940,799.942,3
...,...,...,...,...,...,...,...,...
367,317,2456.39,810.375,3319.490,1553.47,460.005,3015.860,4
368,318,2457.82,804.907,3325.780,1553.04,451.270,3020.530,4
369,319,2459.34,799.367,3332.510,1552.58,442.535,3025.220,4
370,320,2460.89,793.711,3339.640,1552.08,433.801,3029.920,4


In [12]:
df.to_csv(os.path.join(tmpdir, "channel_LUT.csv"), index=False)

In [13]:
sk = skeletonization.generate_skeleton_image(coords[:, 100:],
                             "/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                             10)

In [14]:
sk = sk.astype("uint8")
save_nifti(sk, os.path.join(tmpdir, "probe_image.nii.gz"), 1, 1, 1)

In [15]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/P190/indexing', 'deigo:/bucket/ReiterU/DBS/data/20220526_P190_laqueus_DiI_L-F11/analysis'], returncode=0)